In [1]:
import image_prediction
from importlib import reload

In [2]:
reload(image_prediction)

<module 'image_prediction' from '/home/mark/projects/daiid-app/AI_Image_Detection/image_prediction.py'>

In [3]:
print(image_prediction.is_image_AI_generated('images/dog.jpg'))

/home/mark/projects/daiid-app/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.9997029900550842


In [4]:
print(image_prediction.is_image_AI_generated('images/aidog.jpeg'))

0.9999960660934448


In [5]:
print(image_prediction.is_image_AI_generated('images/ai_1.jpeg'))

0.9871503114700317


In [6]:
print(image_prediction.is_image_AI_generated('images/ai_2.jpeg'))

0.9901397824287415


In [7]:
print(image_prediction.is_image_AI_generated('images/real_1.jpg')) 

0.03394946828484535


In [8]:
print(image_prediction.is_image_AI_generated('images/real_2.jpg')) 

0.9990993738174438


In [9]:
print(image_prediction.is_image_AI_generated('images/real_3.jpg'))

0.9983526468276978


In [10]:
print(image_prediction.is_image_AI_generated('images/NYT_test.webp'))

0.5126586556434631


In [11]:
print(image_prediction.is_image_AI_generated('images/NYT_test_2.webp'))

0.9896078705787659
